#### Find the base directory

In [19]:
import os
base = os.getcwd().replace('\\', '/').replace('/examples', '')
static = os.sep.join([base, 'exatomic', '_static'])

#### Provided the outputs are good and the parser is implemented to handle the specifics of the input arguments, this is what the API looks like for orbital viewing
<li> First you parse the output (must contain full basis set info and full C matrix)
<li> The basis set info is accessible as a gaussian_basis_set attribute and the C matrix as a momatrix attribute
<li> It is an Editor object, which must be converted to a Universe object by .to_universe()
<li> The Universe has an add_molecular_orbitals method which by default requires no arguments
<li> add_molecular_orbitals optionally can accept the following arguments:
<li> params (tup/list): specify numerical grid parameters (eg. to make high resolution surfaces)
<li> vector (int/list/range): specify the molecular orbitals you would like to view
<li> Don't forget to see the actual docstrings for each function

## Gaussian API

In [2]:
gaussian = exatomic.gaussian.Output(os.sep.join([static, 'gaussian.uo2.out'])).to_universe()

In [3]:
gaussian.add_molecular_orbitals(vector=range(20))

Compiling basis functions, may take a while.
Took 15.04s to compile basis functions with 33172 characters, 250 primitives and 141 contracted functions


#### Visually inspect in the widget (access the surfaces in fields not orbitals) -- the widget is general purpose, only the information provided in the universe will behave as expected. So don't be surprised if many options don't do anything, that is expected. You can however click on display and check spheres for a better looking ball-and-stick model 

In [9]:
gaussian

#### Say you want to put surface 12 in a publication
Image will be saved in the same location as the notebook

Specify higher resolution field parameters: (rmin, rmax, nr) so minimum in x,y,z, maximum in x,y,z and number of points

Be careful with nr, remember it is nr^3 points and may be memory-intensive 

In [4]:
gaussian.update_molecular_orbitals(field_params=(-6, 6, 100), vector=12)

(-6, 6, 150)
<class 'tuple'>


#### Orient the scene the way you want it to look then click the save image bar on the widget GUI controls

In [5]:
gaussian

## Molcas API

#### Everything done above can be done with other supported QM codes, just access the appropriate Editor objects

In [6]:
molcas = exatomic.molcas.Output(os.sep.join([static, 'molcas.uo2.out'])).to_universe()

#### Molcas prints out Orb files which contain full C matrix coefficients, just supply the *Orb file you want

In [7]:
molcas.momatrix = exatomic.molcas.Orb(os.sep.join([static, 'molcas.uo2.orb'])).momatrix

#### This was an all-electron calculation and didn't use an ECP, so the vector indices are different than above but roughly correspond to similar orbitals, eg. the first d orbital is hidden in the ECP above but not here (and as it uses an ANO generally contracted basis, evaluates many more primitives, taking a bit longer)

In [11]:
molcas.add_molecular_orbitals(vector=range(40, 60))

Compiling basis functions, may take a while.
Took 34.91s to compile basis functions with 133157 characters, 1374 primitives and 69 contracted functions


In [15]:
molcas

## NWChem API

#### Hopefully it is becoming clear now

In [16]:
nwchem = exatomic.nwchem.Output(os.sep.join([static, 'nwchem.uo2.out'])).to_universe()

In [17]:
nwchem.add_molecular_orbitals(vector=range(20))

Compiling basis functions, may take a while.
Took 18.91s to compile basis functions with 33042 characters, 258 primitives and 139 contracted functions


#### Still some bugs to work out, the basis function specification looks messed up for the 2 U f pi bonding orbitals (vectors 8 and 9)
#### but the point is all 3 QM codes shown here use wildly different basis function ordering schemes and the point of this code is to generalize and make agnostic the internal workings of those codes by well defined data frame specifications of all of this data

In [18]:
nwchem

## Developer details

#### If you are interested in helping, that's great!

The Editor objects deal with file IO and contain parse_methods specific to each dataframe.

In order to add_molecular_orbitals, the Editor must parse the Atom, GaussianBasisSet, BasisSetOrder and MOMatrix dataframes.

The requirements for these dataframes are specified in their docstrings.

The actual interface with the widget occurs on the Universe object, so the Editor must be converted to a Universe.

Therefore, it makes sense to subclass the exatomic.Editor to gain access to the .to_universe() method.